# 清理超大 Highres 图片及对应 Lowres

为避免推理时显存溢出，应该从 **highres** 端控制：删除 **highres 总像素超过 25.6M**（即 1600×1000×4×4，对应 4× 放大后的 LR 为 1.6M 像素）的图片，并同时删除其对应的 lowres 图。

配对规则：与 `degradation_pipeline.ipynb` / `train_esrgan_independent.ipynb` 一致，lowres 与 highres 通过 **相同文件名** 一一对应（如 `372159.jpg` 在 `dataset/lowres/original/` 与 `dataset/highres/original/` 中各一份）。

> 注意：这个 notebook 的旧版本已经按 lowres>1.6M 删除过一遍，这里是 **新的规则：直接按 highres>25.6M 检查并删除**。

In [1]:
import os
import glob

# 与 degradation_pipeline / 训练脚本保持一致
LR_DIR = 'dataset/lowres/original'
HR_DIR = 'dataset/highres/original'

# Highres 总像素上限：超过则删除（1600*1000*4*4 = 25.6M）
MAX_PIXELS = 25_600_000

In [2]:
from PIL import Image

def get_image_pixels(path):
    """获取图片总像素（宽*高），失败返回 None。"""
    try:
        with Image.open(path) as im:
            w, h = im.size
            return w * h
    except Exception as e:
        print(f"  无法读取: {path} -> {e}")
        return None

# 获取所有 highres 图片路径（与 degradation 一致：jpg/jpeg/png）
hr_paths = sorted(glob.glob(os.path.join(HR_DIR, '*.[jp][pn]*g')))
print(f"Highres 目录中共 {len(hr_paths)} 张图片。")

to_remove = []  # [(lr_path, hr_path, pixels), ...]
for hr_path in hr_paths:
    pixels = get_image_pixels(hr_path)
    if pixels is None:
        continue
    if pixels > MAX_PIXELS:
        filename = os.path.basename(hr_path)
        lr_path = os.path.join(LR_DIR, filename)
        to_remove.append((lr_path, hr_path, pixels))

print(f"其中 highres 总像素 > {MAX_PIXELS:,} 的共有 {len(to_remove)} 张，将连同对应 lowres 一并删除。")

Lowres 目录中共 460 张图片。
其中总像素 > 1,600,000 的共有 26 张，将连同对应 highres 一并删除。


In [3]:
if not to_remove:
    print("没有需要删除的图片。")
else:
    print("待删除列表（highres 路径 | 总像素）：")
    for lr_path, hr_path, pixels in to_remove:
        name = os.path.basename(hr_path)
        with Image.open(hr_path) as im:
            w, h = im.size
        print(f"  {name}: {w}*{h} = {pixels:,} 像素")

待删除列表（lowres 路径 | 总像素）：
  372159.jpg: 2362*1574 = 3,717,788 像素
  372172.jpg: 1792*1149 = 2,059,008 像素
  373425.jpg: 1754*1240 = 2,174,960 像素
  373549.jpg: 1968*1389 = 2,733,552 像素
  373680.jpg: 2223*1182 = 2,627,586 像素
  374729.jpg: 3018*1083 = 3,268,494 像素
  375282.jpg: 2000*1264 = 2,528,000 像素
  375382.jpg: 1920*1080 = 2,073,600 像素
  378937.jpg: 2048*1268 = 2,596,864 像素
  379138.jpg: 2052*1290 = 2,647,080 像素
  379479.jpg: 1750*1325 = 2,318,750 像素
  381639.jpg: 1790*1158 = 2,072,820 像素
  382338.jpg: 1750*1147 = 2,007,250 像素
  384731.jpg: 1578*1116 = 1,761,048 像素
  386490.jpg: 1920*1080 = 2,073,600 像素
  386651.jpg: 2315*930 = 2,152,950 像素
  387393.jpg: 2400*1350 = 3,240,000 像素
  387402.jpg: 2400*1350 = 3,240,000 像素
  387485.jpg: 2000*1075 = 2,150,000 像素
  388375.jpg: 2802*1250 = 3,502,500 像素
  393567.jpg: 1920*1080 = 2,073,600 像素
  394659.jpg: 1774*945 = 1,676,430 像素
  395724.jpg: 1920*1080 = 2,073,600 像素
  396358.jpg: 2166*1251 = 2,709,666 像素
  397954.jpg: 2126*1195 = 2,540,570 像素
  3

In [4]:
deleted_lr = 0
deleted_hr = 0

for lr_path, hr_path, pixels in to_remove:
    if os.path.isfile(lr_path):
        os.remove(lr_path)
        deleted_lr += 1
        print(f"已删除 lowres:  {os.path.basename(lr_path)}")
    if os.path.isfile(hr_path):
        os.remove(hr_path)
        deleted_hr += 1
        print(f"已删除 highres: {os.path.basename(hr_path)}")
    else:
        print(f"  警告: 未找到对应 highres: {os.path.basename(hr_path)}")

print(f"\n完成。共删除 lowres {deleted_lr} 张、highres {deleted_hr} 张。")

已删除 lowres:  372159.jpg
已删除 highres: 372159.jpg
已删除 lowres:  372172.jpg
已删除 highres: 372172.jpg
已删除 lowres:  373425.jpg
已删除 highres: 373425.jpg
已删除 lowres:  373549.jpg
已删除 highres: 373549.jpg
已删除 lowres:  373680.jpg
已删除 highres: 373680.jpg
已删除 lowres:  374729.jpg
  警告: 未找到对应 highres: 374729.jpg
已删除 lowres:  375282.jpg
已删除 highres: 375282.jpg
已删除 lowres:  375382.jpg
已删除 highres: 375382.jpg
已删除 lowres:  378937.jpg
  警告: 未找到对应 highres: 378937.jpg
已删除 lowres:  379138.jpg
已删除 highres: 379138.jpg
已删除 lowres:  379479.jpg
  警告: 未找到对应 highres: 379479.jpg
已删除 lowres:  381639.jpg
  警告: 未找到对应 highres: 381639.jpg
已删除 lowres:  382338.jpg
  警告: 未找到对应 highres: 382338.jpg
已删除 lowres:  384731.jpg
  警告: 未找到对应 highres: 384731.jpg
已删除 lowres:  386490.jpg
  警告: 未找到对应 highres: 386490.jpg
已删除 lowres:  386651.jpg
已删除 highres: 386651.jpg
已删除 lowres:  387393.jpg
已删除 highres: 387393.jpg
已删除 lowres:  387402.jpg
已删除 highres: 387402.jpg
已删除 lowres:  387485.jpg
已删除 highres: 387485.jpg
已删除 lowres:  388375.jpg
已删除 high